In [1]:
import json
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('fulldataset.csv')

In [9]:
df.head()

,Unnamed: 0,InitialLatitude,InitialLongitude,FireDiscoveryDateTime,date,tempature0,wind0,precipitation0,soilmoisture0,tempature1,...,tempature3,wind3,precipitation3,soilmoisture3,tempature4,wind4,precipitation4,soilmoisture4,FireOutDateTime,fire_status
0,0,47.63531,-117.3354,2017/08/02 20:23:32,2017-08-02,284.1,4.088354,0.0,25.51,281.6,...,288.1,3.848339,0.1044,25.50,288.5,2.759575,0.0,25.37,2017/08/06 17:29:00+00,1
1,1,47.67908,-117.6248,2019/06/05 20:47:04,2019-06-05,284.1,4.088354,0.0,25.51,281.6,...,288.1,3.848339,0.1044,25.50,288.5,2.759575,0.0,25.37,2019/06/10 16:44:00+00,1
2,2,48.21964,-117.0441,2018/06/23 01:52:57,2018-06-23,281.9,4.074414,0.0,28.06,279.0,...,285.7,3.857150,0.8760,27.70,286.5,2.588716,0.0,27.58,2018/07/01 20:55:59+00,1
3,3,46.15799,-123.3071,2019/03/20 00:04:59,2019-03-20,288.3,4.046970,0.0,29.25,287.7,...,291.0,4.442897,0.0000,29.45,291.2,3.995460,0.0,29.73,2019/09/05 20:55:00+00,1
4,4,47.30909,-122.8772,2019/04/29 19:56:00,2019-04-29,290.4,3.782720,0.0,21.13,289.7,...,292.6,4.630771,0.0288,21.45,292.5,3.682093,0.0,21.86,2019/05/14 20:35:59+00,1


In [5]:
name_list=[]
for i in range(5):
    name_list.append("tempature"+str(i))
    name_list.append("wind"+str(i))
    name_list.append("precipitation"+str(i))
    name_list.append("soilmoisture"+str(i))

In [7]:
df_input=df[name_list]

In [10]:
df_output=df['fire_status']

In [12]:
inputs=df_input.to_numpy()

In [13]:
outputs=df_output.to_numpy()

In [14]:
inputs.shape

(9590, 20)

In [16]:
outputs=outputs.reshape(len(outputs),1)

In [17]:
outputs.shape

(9590, 1)

In [20]:
from sklearn.model_selection import train_test_split


In [21]:
x, x_test, y, y_test = train_test_split(inputs, outputs, test_size=0.2, shuffle=True)

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [23]:
def Model(input_dim, activation, num_class):
    model = Sequential()
    
    
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation(activation))

    model.add(Dropout(0.5))

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.5))

    model.add(Dense(512))
    model.add(BatchNormalization())    
    model.add(Activation(activation))
    model.add(Dropout(0.5))

    
    model.add(Dense(num_class))    
    model.add(Activation('linear'))
    model.compile(loss='binary_crossentropy',
                      optimizer=optimizers.Adam(lr = 0.001),
                      metrics=['accuracy']
                )
    return model

In [27]:
input_dim = inputs.shape[1]
activation = 'relu'
classes = 1
model = Model(input_dim=input_dim, activation=activation, num_class=classes)


In [28]:
history = model.fit(x=x, y=y, batch_size=64, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
120/120 [==============================] - 4s 19ms/step - loss: 1.1702 - accuracy: 0.8942 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 2/100
120/120 [==============================] - 2s 19ms/step - loss: 0.6660 - accuracy: 0.9440 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 3/100
120/120 [==============================] - 2s 18ms/step - loss: 0.9221 - accuracy: 0.9241 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 4/100
120/120 [==============================] - 2s 18ms/step - loss: 0.7612 - accuracy: 0.9395 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 5/100
120/120 [==============================] - 2s 18ms/step - loss: 1.2272 - accuracy: 0.9067 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 6/100
120/120 [==============================] - 2s 17ms/step - loss: 0.8706 - accuracy: 0.9297 - val_loss: 5.6858 - val_accuracy: 0.6314
Epoch 7/100
120/120 [==============================] - 2s 19ms/step - loss: 0.6232 - accuracy: 0.9489 - val_loss: 5.6858 - val_accuracy:

In [29]:
model.save('./fire_model_1.0',save_format='tf')

INFO:tensorflow:Assets written to: ./fire_model_1.0/assets
